In [1]:
import numpy as np
from microtorch import Tensor
from microtorch import nn

# XOR

In [2]:
x = Tensor([
    [0.0, 0.0],
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0]
])

y = Tensor([
    [0.0],
    [1.0],
    [1.0],
    [0.0],
])

In [18]:
model = nn.Sequential(
    nn.Linear(2, 16),
    nn.LeakyReLU(0.01),
    nn.Linear(16, 16),
    nn.LeakyReLU(0.01),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

In [19]:
def optimize(params, lr=0.001):
    for p in params:
        p.data -= lr * p.grad
        
def zero_grad(*params):
    for p in params:
        p.zero_grad()

In [20]:
for i in range(1000):
    loss = Tensor.l2(model(x), y)
    
    zero_grad(*model.params(), x)
    
    loss.backward()
    
    optimize(model.params(), lr=0.1)

In [21]:
pred = model(x)
pred

Tensor([[0.0200307 ],
        [0.98323649],
        [0.97924898],
        [0.01713272]])